In [ ]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import time
import re
import os
import shutil

In [ ]:
# Folders os cpf/cnpjs that already exist
folders_cpf_cnpjs = [f for f in os.listdir() if re.match('^[0-9]+$',f)]

# List of cpf cnpj to execute
l_cpf_cnpj = ['00564427934','45171025991','62194860134','38811855187','80866387153']

# Create folder
[os.mkdir(c) for c in l_cpf_cnpj if not c in folders_cpf_cnpjs]

# Folders os cpf/cnpjs that already exist
folders_cpf_cnpjs = [f for f in os.listdir() if re.match('^[0-9]+$',f)]

In [ ]:
# Driver to start chrome
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
prefs = {"profile.default_content_settings.popups": 0,"download.default_directory": r"C:\Users\lukin\Desktop\Anoregmt\\", "directory_upgrade": True, "plugins.always_open_pdf_externally": True }
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path='C:/Users/lukin/Downloads/chromedriver', options=options)

# Get site
driver.get('https://app.anoregmt.org.br/#/login')

In [ ]:
# Define credentials
user = ''
password = ''

# Credentials
user_we = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[2]/label[1]/div/div/div/input')
user_we.send_keys(user)
password_we = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[2]/label[2]/div/div[1]/div[1]/input')
password_we.send_keys(password)

# Access page
access = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[3]/div/div[1]/button/span[2]/span')
access.click()

# Wait 5 seconds
time.sleep(5)

# Access "Pesquisa Dinamica"
pesquisa_dinamica = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[1]/aside/div/div[1]/div[1]/div/div[2]/div[3]/div[3]/div')
pesquisa_dinamica.click()

# Wait 3 seconds
time.sleep(3)

# Input to cpf/cnpj
cpf_cnpj = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[2]/div/div/div/label/div/div[1]/div[2]/input')
cpf_cnpj.send_keys(l_cpf_cnpj[4])

# Click on "Pesquisar"
pesquisar = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/label/div/div[1]/div[3]/button/span[2]/span/span')
pesquisar.click()

In [ ]:
# Result from search
result = driver.find_elements_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[1]/table/tbody/tr/td[1]')

# Result's text
result_text = np.array([r.text for r in result])

# Index to get the right button position
index_result = np.where(result_text == 'Registro de Imóveis')[0]

if(len(index_result) != 0):
    # Click on "Registro de Imóveis"
    result[index_result[0]].click()

In [ ]:
# Get Number of pages
number_of_pages = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[2]/span')
number_of_pages = int(number_of_pages.text.split(' ')[2])

# Expand all pages
for i in range(number_of_pages-1):
    
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Click on "Ver Mais" to open other registers
    ver_mais = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[3]/button/span[2]/span/span')
    ver_mais.click()
    
    # Wait 3 seconds
    time.sleep(3)
    
# Result of search
result = driver.find_elements_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[1]/table/tbody/tr/td[1]')

# Scroll to the page's start
driver.execute_script("window.scrollTo(0, 0);")

In [ ]:
# For each register
for i in range(5,20):
    try:
        # Click to open a window
        result[i].click()

        # Wait 5 seconds
        time.sleep(5)

        # Click on button "visualizar"
        visualizar = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/div[4]/button[1]/span[2]/span/span')
        visualizar.click()

        # Wait 5 seconds
        time.sleep(5)

        if(len(driver.window_handles) > 1):
            # Switch to new tab created
            driver.switch_to.window(driver.window_handles[-1])

            # Close this new tab
            driver.close()

            # Back to original tab
            driver.switch_to.window(driver.window_handles[0])

        # Click on "X" button
        exit_btn = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/div[1]/button/span[2]/span/i')
        exit_btn.click()

        # Scroll to next element
        driver.execute_script("arguments[0].scrollIntoView(true);", result[i]);

        # Wait 3 seconds
        time.sleep(3)
    except:
        time.sleep(5)
        print(i)

In [ ]:
# Move downloaded pdf to right folder
pdfs = [f for f in os.listdir() if re.match('^.*\.pdf',f)]
for p in pdfs:
    shutil.move(os.getcwd() + '\\' + p,os.getcwd() + '\\' + l_cpf_cnpj[4] + '\\' + p)